<a href="https://colab.research.google.com/github/robertmaxwilliams/talking-statues/blob/master/colab/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>